In [ ]:
import discord
import re
from fuzzywuzzy import fuzz
import nest_asyncio
import asyncio
import os
from dotenv import load_dotenv

load_dotenv()  # charge le fichier .env

token = os.getenv("DISCORD_TOKEN")
if token is None:
    raise ValueError("⚠️ La variable d'environnement DISCORD_TOKEN n'est pas définie.")

print("Token chargé avec succès ✅")

# note d'intention à  faire + dictionnaire d'intents avec des mots-clés Artiste/Lyrics
# Appliquer nest_asyncio pour corriger la boucle d'événements de Jupyter
nest_asyncio.apply()

# TOKEN DU BOT DISCORD
  # ⚠️ remplace ça par une variable d’environnement dans un vrai projet

# Intents Discord (accès aux messages, membres, etc.)
intents_discord = discord.Intents.all()
intents_discord.members = True

client = discord.Client(intents=intents_discord)

# Dictionnaire d’intents
intents = {
    "chanter": {
        "Sardou": {
            "titre": "Les Lacs du Connemara",
            "paroles": "C’est en Irlande qu’il est né, entre Galway et Limerick,\nDans un coin sauvage de terre brûlée...\nAux lacs du Connemara !"
        },
        "Renaud": {
            "titre": "Mistral gagnant",
            "paroles": "À m'asseoir sur un banc cinq minutes avec toi,\nEt regarder les gens tant qu'y en a..."
        },
        "Calogero": {
            "titre": "En apesanteur",
            "paroles": "Et si je te disais que rien ne m’empêche de rêver,\nMême en apesanteur..."
        },
        "Steve": {
            "titre": "Le Poasson Steve",
            "paroles": "Steve, le poisson, nage dans l'océan,\nAvec ses amis, il est toujours content..."
        },
        "Dalida": {
            "titre": "Gigi l'Amoroso",
            "paroles": "Gigi l'Amoroso, croqueur d'amour, l'œil de velours,\nComme une caresse..."
        },
        "Johnny": {
            "titre": "Que je t'aime",
            "paroles": "Quand tes cheveux s'étalent,\nComme un soleil d'été..."
        },
        "Brel": {
            "titre": "Ne me quitte pas",
            "paroles": "Ne me quitte pas, il faut oublier,\nTout peut s'oublier qui s'enfuit déjà..."
        },
        "Aznavour": {
            "titre": "La Bohème",
            "paroles": "La Bohème, la Bohème,\nÇa voulait dire, on est heureux..."
        }
    },
    "au_revoir": ["au revoir", "bye", "ciao", "à bientôt"],
    "remerciement": ["merci", "thanks", "merci beaucoup", "thx"],
    "aide": ["aide", "support", "besoin d'aide", "je comprends pas"]
}


def get_intent(user_input, intents, threshold=80):
    user_input = user_input.lower()
    
    # Cas spécial : intent "chanter" avec un artiste
    if "chante" in user_input or "chanson" in user_input:
        for artiste in intents.get("chanter", {}):
            if artiste.lower() in user_input:
                return "chanter", artiste
        return "chanter", None  # "chanter" sans artiste reconnu

    # Autres intents classiques
    best_score = 0
    best_intent = "intent_inconnu"
    for intent, keywords in intents.items():
        if isinstance(keywords, dict):  # Skip dictionnaires imbriqués
            continue
        for keyword in keywords:
            score = fuzz.partial_ratio(user_input, keyword.lower())
            if score > best_score and score >= threshold:
                best_score = score
                best_intent = intent

    return best_intent, None

# Event quand le bot est prêt
@client.event
async def on_ready():
    print("✅ Le bot est prêt")

# Event quand un message est reçu
@client.event
@client.event
async def on_message(message):
    if message.author == client.user:
        return

    mess = message.content.lower()

    if mess.startswith('steve'):
        pattern = r'^steve\s*'
        cleaned_message = re.sub(pattern, '', mess, flags=re.IGNORECASE)
        detected_intent, artiste = get_intent(cleaned_message, intents)

        if detected_intent == "chanter":
            if artiste and artiste in intents["chanter"]:
                chanson = intents["chanter"][artiste]
                await message.channel.send(
                    f"🎵 **{artiste} – {chanson['titre']}**\n\n{chanson['paroles']}"
                )
            else:
                await message.channel.send("Je ne connais pas encore cet artiste 😥.")
        else:
            await message.channel.send(f"Intent reconnu : **{detected_intent}**")

# Lancer le bot sans utiliser client.run() (car on est dans Jupyter)
async def start_bot():
    await client.start(token)

await start_bot()


Token chargé avec succès ✅
✅ Le bot est prêt
